In [9]:
# 1. 라이브러리 및 환경 설정
from typing import TypedDict
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnableLambda
import os
import requests
import datetime
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
# 2. 상태 정의
class QAState(TypedDict):
    question: str
    route_label: str
    retrieved: str
    answer: str

In [11]:
# 3. LLM 기반 라우팅 판단
def llm_router_update(state: QAState) -> QAState:
    prompt = f"""
    사용자의 질문을 보고 아래 중 어떤 유형인지 판단하세요:

    - general: 일반 지식이나 개념
    - web: 최근 뉴스, 최신 정보
    - db: 회의록, 내부 데이터 요약

    질문: "{state['question']}"

    위 3가지 중 하나의 단어(general/web/db)만 반환하세요.
    """
    route = llm.invoke(prompt).content.strip().lower()
    print(f"🔀 [LLM 라우팅 판단] → {route}")
    state["route_label"] = route
    return state

def return_route_key(state: QAState) -> str:
    return state["route_label"]

In [12]:
# 4. 네이버 검색어 트렌드 API 연동
def fetch_search_trend(query="AI"):
    url = "https://openapi.naver.com/v1/datalab/search"
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET"),
        "Content-Type": "application/json"
    }

    end = datetime.date.today()
    start = end - datetime.timedelta(days=7)

    body = {
        "startDate": start.strftime("%Y-%m-%d"),
        "endDate": end.strftime("%Y-%m-%d"),
        "timeUnit": "date",
        "keywordGroups": [
            {"groupName": query, "keywords": [query]}
        ]
    }

    response = requests.post(url, headers=headers, json=body)
    data = response.json()

    results = data.get("results", [])
    if not results or not results[0].get("data"):
        return "[검색 결과 없음]"

    trends = results[0]["data"]
    return "\n".join([f"{item['period']}: {item['ratio']}%" for item in trends])

In [17]:
# 5. 실행 노드 정의
def general_llm(state: QAState) -> QAState:
    response = llm.invoke(f"질문: {state['question']}\n\n위 질문에 답해주세요.")
    state["answer"] = response.content.strip()
    return state

def web_search(state: QAState) -> QAState:
    def extract_keyword(q: str) -> str:
        for kw in ["AI", "경제", "정치", "한국", "북한", "날씨", "연예", "스포츠"]:
            if kw in q:
                return kw
        return "스포츠"

    keyword = extract_keyword(state["question"])

    try:
        raw_trend = fetch_search_trend(keyword)
        if "[검색 결과 없음]" in raw_trend:
            summary = "검색어 트렌드 결과가 충분하지 않아 요약할 수 없습니다."
        else:
            prompt = f"""
            다음은 키워드 '{keyword}'에 대한 최근 7일간의 검색 트렌드입니다:
            {raw_trend}

            이 키워드가 최근 왜 관심을 받았는지, 뉴스나 사회적 맥락을 추측하여 요약해 주세요.
            """
            response = llm.invoke(prompt)
            summary = response.content.strip()
    except Exception as e:
        summary = f"[검색 트렌드 요약 실패] {e}"

    state["retrieved"] = summary
    return state

def db_search(state: QAState) -> QAState:
    state["retrieved"] = "[DB 검색 결과] 회의 내용을 요약했습니다."
    return state

def generate_answer(state: QAState) -> QAState:
    if state.get("retrieved"):
        state["answer"] = f"[검색 기반 응답] {state['retrieved']}"
    return state

In [18]:
# 6. LangGraph 정의
def build_news_router_graph():
    builder = StateGraph(QAState)

    builder.add_node("judge_route", RunnableLambda(llm_router_update))
    builder.add_node("general", RunnableLambda(general_llm))
    builder.add_node("web", RunnableLambda(web_search))
    builder.add_node("db", RunnableLambda(db_search))
    builder.add_node("generate", RunnableLambda(generate_answer))

    builder.set_entry_point("judge_route")

    builder.add_conditional_edges(
        "judge_route",
        return_route_key,
        {
            "general": "general",
            "web": "web",
            "db": "db"
        }
    )

    builder.add_edge("general", "generate")
    builder.add_edge("web", "generate")
    builder.add_edge("db", "generate")

    builder.set_finish_point("generate")

    return builder.compile()

In [19]:
# 7. 실행 테스트
graph = build_news_router_graph()

questions = [
    "에이전트란 무엇인가요?",
    "오늘의 주요 뉴스 알려줘",
    "지난 회의 요약해줘"
]

for q in questions:
    print(f"\n❓ 질문: {q}")
    result = graph.invoke({
        "question": q,
        "route_label": "",
        "retrieved": "",
        "answer": ""
    })
    print(f"✅ 응답: {result['answer']}")


❓ 질문: 에이전트란 무엇인가요?
🔀 [LLM 라우팅 판단] → general
✅ 응답: "에이전트"라는 용어는 다양한 맥락에서 사용될 수 있지만, 일반적으로 다음과 같은 의미를 가집니다.

1. **일반적인 의미**: 에이전트는 어떤 일을 대신 수행하는 대리인이나 대행자를 의미합니다. 예를 들어, 부동산 에이전트는 고객을 대신하여 부동산 거래를 중개하는 역할을 합니다.

2. **컴퓨터 과학 및 인공지능**: 에이전트는 특정 환경에서 자율적으로 행동하고 결정을 내릴 수 있는 프로그램이나 시스템을 의미합니다. 예를 들어, 소프트웨어 에이전트는 사용자의 요구를 충족시키기 위해 정보를 수집하고 처리하는 역할을 합니다.

3. **비즈니스 및 마케팅**: 에이전트는 특정 제품이나 서비스를 홍보하고 판매하는 역할을 하는 사람이나 회사를 지칭할 수 있습니다. 이들은 제조업체와 소비자 사이의 중개 역할을 합니다.

4. **법률**: 법률에서 에이전트는 다른 사람(주로 'principal')을 대신하여 법적 행위를 수행할 수 있는 권한을 가진 사람을 의미합니다.

따라서 "에이전트"라는 용어는 그 사용되는 분야에 따라 다양한 의미를 가질 수 있습니다.

❓ 질문: 오늘의 주요 뉴스 알려줘
🔀 [LLM 라우팅 판단] → web
✅ 응답: [검색 기반 응답] 최근 '스포츠' 키워드에 대한 검색 트렌드가 급증한 이유는 여러 가지 사회적 및 뉴스적 맥락에서 추측할 수 있습니다. 

1. **스포츠 이벤트**: 2025년 3월 22일에 100%의 검색 비율을 기록한 것은 주요 스포츠 이벤트가 있었음을 시사합니다. 예를 들어, 국제적인 스포츠 대회(예: 월드컵, 올림픽, 챔피언스리그 등)나 인기 있는 리그의 결승전이 개최되었을 가능성이 높습니다.

2. **스타 선수의 이슈**: 특정 스타 선수의 이적, 부상, 혹은 중요한 경기에서의 활약 등이 뉴스에 보도되면서 관심이 집중되었을 수 있습니다. 이러한 사건들은 팬들의 검색을 유도할 수 있습니다.

3. **사회적 이슈**: 스포츠